# Using GloVe Embedding

© Data Trainers LLC. GPL v 3.0.

**Author:** Axel Sirota


In this notebook we will leverage Standford's GloVe vectors which is a pretrained embedding on 1.4B Tweets.

Take it easy and pay attention to the main differences with before, and the non-trainable parameters. Finally, check how accurate the results now are.
You can run this lab both locally or in Colab.

- To run in Colab just go to `https://colab.research.google.com`, sign-in and you upload this notebook. Colab has GPU access for free.
- To run locally just run `jupyter notebook` and access the notebook in this lab. You would need to first install the requirements in `requirements.txt`

Follow the instructions. Good luck!


In [ ]:
!nvidia-smi

In [ ]:
!pip install --upgrade  textblob gensim pytorch-nlp swifter

In [ ]:
import multiprocessing
import torch
import torch.nn as nn
import torch.optim as optim
import itertools
import sys
from textblob import TextBlob, Word
import numpy as np
import random
import os
import pandas as pd
import gensim
import warnings
import nltk

TRACE = False  # Setting to true is useful when debugging to know which device is being used
embedding_dim = 100
epochs=10

def set_seeds_and_trace():
  os.environ['PYTHONHASHSEED'] = '0'
  np.random.seed(42)
  random.seed(42)


set_seeds_and_trace()
warnings.filterwarnings('ignore')
nltk.download('punkt')
textblob_tokenizer = lambda x: TextBlob(x).words

In [ ]:
%%writefile get_data.sh
if [ ! -f yelp.csv ]; then
  wget -O yelp.csv https://www.dropbox.com/s/xds4lua69b7okw8/yelp.csv?dl=0
fi

if [ ! -f glove.6B.100d.txt ]; then
  wget -O glove.6B.100d.txt https://www.dropbox.com/s/dl1vswq2sz5f1ws/glove.6B.100d.txt?dl=0
fi

In [ ]:
!bash get_data.sh

In [ ]:
path = './yelp.csv'
yelp = pd.read_csv(path)
# Create a new DataFrame that only contains the 5-star and 1-star reviews to have extremes.
yelp_best_worst = yelp[(yelp.stars==5) | (yelp.stars==1)]
X = yelp_best_worst.text
y = yelp_best_worst.stars.map({1:0, 5:1})

In [ ]:
corpus = [sentence for sentence in X.values if type(sentence) == str and len(TextBlob(sentence).words) > 3]

In [ ]:
path_to_glove_file = "./glove.6B.100d.txt"
embeddings_index = {}
# Construct a function that fills the embedding_index dict for every word in the GloVe file with its coefficients.
# HELP: For that iterate over the Glove file (hint: check that file to view its structure first!), split the word from the numbers, and populate the dictionary with the word and the numbers as a numpy array.
# Hint2: check np.fromstring
# FILL

print("Found %s word vectors." % len(embeddings_index))

In [ ]:
import itertools
from torchnlp.encoders import LabelEncoder


list_of_words = list(itertools.chain.from_iterable([sentence.split() for sentence in corpus]))
ids_from_words = LabelEncoder(list_of_words, reserved_labels=['UNK'], unknown_index=0, min_occurrences=1)


# tokenizer = Tokenizer()
# # Use the fit_on_texts method to fit the tokenizer
# tokenizer.fit_on_texts(corpus) # Fill

print(f'Before the tokenizer: {corpus[:1]}')

#Now use the same "trained" tokenizer to convert the corpus from words to IDs with the texts_to_sequences method
tokenized_corpus = [ids_from_words.batch_encode(sentence.split()) for sentence in corpus]

print(f'After the tokenizer: {tokenized_corpus[:1]}')

In [ ]:

def ids_from_text(text):
  return ids_from_words.batch_encode(text)

def text_from_ids(ids):
  return ids_from_words.batch_decode(ids)


In [ ]:
hits = 0
misses = 0

embedding_matrix = np.zeros((vocab_size, embedding_dim))

# Create a loop such that for every word in the vocabulary, if it exists in the Glove embedding, then set for that word (that means that index) the tensor of the Glove Embedding. Otherwise fill with 0
# FILL

# In the end, the embedding matrix should have, for every word of our tokenizer that exists in GloVe, the tensor representation of that word


In [ ]:
def pad_sequence_of_tokens(x, maxlen, unk_token='UNK'):
  if len(x)<maxlen:
    x.extend([unk_token]*(maxlen-len(x)))
  return x

In [ ]:
# This is the algorithmic part of batching the dataset and yielding the window of words and expected middle word for each bacth as a generator.
def create_context_target_pairs(texts, context_size):
    data = []
    for text in texts:
        tokens = text.split()
        for i, word in enumerate(tokens):
            start = max(0, i - context_size)
            end = min(len(tokens), i + context_size + 1)
            context = pad_sequence_of_tokens([tokens[j] for j in range(start, end) if j != i], maxlen=4)
            target = ids_from_words.token_to_index[word]
            context_indices = [ids_from_words.token_to_index[w] for w in context]
            context_indices.append(target)
            data.append(torch.Tensor(context_indices))
    return data

In [ ]:
data = create_context_target_pairs(corpus[:500], 2)

In [ ]:
data = torch.stack(data)

In [ ]:
X = data[:, :4].to(torch.long)
y = data[:, 4].to(torch.long)



In [ ]:

class CBOW(nn.Module):
    def __init__(self, vocab_size, embedding_dim, context_size):
        super(CBOW, self).__init__()
        self.embeddings = None # Instantiate the Embedding with the matrix we created
        # Linear layer to act as the hidden layer
        self.linear1 = None # Use the linear layer as before
        # Linear layer to predict the center word
        self.linear2 = None # Use the linear layer as before

    def forward(self, inputs):
        pass  # Implement

In [ ]:
def train_cbow(X, y, model, loss_function, optimizer, epochs):
    for epoch in range(epochs):
        total_loss = 0

        # Step 1. Recall that torch *accumulates* gradients. Before passing in a new instance,
        # you need to zero out the gradients from the old instance
        optimizer.zero_grad()

        # Step 2. Run the forward pass, getting log probabilities over next words
        log_probs = model(X)

        # Step 3. Compute your loss function. (Again, Torch wants the target
        # word wrapped in a tensor)
        loss = loss_function(log_probs, y)

        # Step 4. Do the backward pass and update the gradient
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        # Print progress
        if (epoch + 1) % 10 == 0:
            print('Epoch: {}, Loss: {:.4f}'.format(epoch + 1, total_loss))
    return model

In [ ]:
context_size=2
embedding_dim=100
vocab_size = len(ids_from_words.vocab)
model = CBOW(vocab_size, embedding_dim, context_size * 2)
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001)

In [ ]:
trained_model = train_cbow(X, y, model, loss_function, optimizer, epochs=1)

In [ ]:
import gensim
from gensim.models.keyedvectors import KeyedVectors

embeddings = trained_model.embeddings.weight.data.cpu().numpy()

# Now, we need to save these embeddings in a format that gensim can understand
# For that, we will use the KeyedVectors instance in gensim

# Instantiate the KeyedVectors with the correct size
kv = KeyedVectors(vector_size=embeddings.shape[1])






In [ ]:
# Add the vectors and their corresponding words to the KeyedVectors instance
kv.add_vectors(ids_from_words.index_to_token, embeddings)

In [ ]:
kv.most_similar(positive=['gasoline'])

In [ ]:
kv.most_similar(negative=['apple'])